# Impulse Responses, Convolution

[return to main page](index.ipynb)

In this notebook we will investigate an unknown digital system.
The only information we have about this system is that it is LTI (linear and time invariant) and that it is defined by the function `tools.blackbox()` in the file [tools.py](tools.py).

To be able to use it, you have to import it:

In [1]:
import tools

After that, you should have a look at the documentation:

In [3]:
tools.blackbox?

In this notebook, we'll try to find out as many things as possible about this system, without looking at its source code.
Later, we'll also have a quick look at a non-linear system.

## Listening to the System

*Exercise:* Load the audio file [data/xmas.wav](data/xmas.wav) and apply the function `tools.blackbox()` to it.

*Exercise:* Listen to the result. Compare the original signal and the resulting signal.
Describe the differences.
What does the system realized by `tools.blackbox()` sound like?

## Obtaining the Impulse Response

An LTI system can be completely described by its impulse response (which may be infinitely long, however).

*Exercise:* Determine the impulse response (or at least an estimation of it) of `tools.blackbox()`.

To do that, use a unit impulse as input to the system.
The resulting output is the impulse response.
To get a meaningful response, append zeros (this is called *zero-padding*) to your unit impulse signal until it has a total length of $\frac{1}{10}$ of a second.

*Exercise:* Plot the impulse response (with the time in seconds along the x-axis).
Note that the amplitude seems to be zero in the end, but it's not!
Zoom into the plot until you can see the non-zero values.

To obtain more insight about the parts of the impulse response with very small amplitudes, we can try logarithmic scaling.

*Exercise:* Plot the impulse response in decibels (i.e. with logarithmic amplitude scaling).
Have a look in [tools.py](tools.py), you might find a useful function there ...

*Exercise:* Use the function [scipy.signal.freqz()](http://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.freqz.html) to calculate the frequency response given the impulse response.

*Exercise:* Plot the magnitude of the frequency response in decibel on a logarithmic frequency axis.

*Exercise:* Try all combinations of logarithmic/linear scaling on the x-/y-axis (that's 4 combinations).

## Naive Convolution

*Exercise:* Write a function called `naive_convolution()` that computes the convolution of two one-dimensional arrays by means of two nested loops according to the equation

$$y[n] = x[n] \ast h[n] = \sum_{m=-\infty}^{\infty} x[m] \cdot h[n-m],$$

where $x$ and $h$ are one-dimensional arrays of finite lengths.
The infinite sum can be changed to a finite sum by assuming that all values before index 0 and all values after the last array element are equal to zero.

Following this assumption, at which indices $n$ does $y[n]$ have its first and last non-zero value?

*Exercise:* Use the function `naive_convolution()` to convolve the audio signal with the impulse response from the previous exercise.

WARNING: The calculation may take a very long time!

*Exercise:* Listen to the result (only if you had the patience to wait for the computation to finish).

*Exercise:* How long is the output signal compared to the input signals?
Does this make sense according to the equation above?
Compare the output signal with the result from the function
`tools.blackbox()`.

## Matrix Convolution

The implementation used in the previous exercise is very inefficient.

*Exercise:* Write a function called `matrix_convolution()`, that does the same thing, but using matrix multiplication.

*Exercise:* Call this function as well (using the same input signal and impulse response as
before) and check if there is a difference in computation time.
Warning: Depending on the order of arguments, this may need huge amounts of memory and bring your computer to a halt.
You should try it first with a small part of the signal (say, the first 1000 samples or so) and then gradually increase the length of the input signal until you know which order is the "right" one.

*Exercise:* Listen to the resulting output signal.
The length of the output signal should be the same as in the previous exercise.

*Exercise:* Switch the two input parameters and check if this has an influence on the computation time.
Warning: See above, you should try this only with a small part of the input signal.

## Fast Convolution

As you know, convolution in the time domain is equivalent to element-wise multiplication in the frequency domain.

*Exercise:* Write a function named `fft_convolution()`, that transforms both inputs to the frequency domain using [numpy.fft.rfft()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.rfft.html), does an element-wise multiplication of the (complex) spectra and transforms the result back with [numpy.fft.irfft()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.irfft.html).
Deliberate whether the result is supposed to be real or complex.
Check the data types and sizes of the intermediate arrays.

*Exercise:* What would happen if you use `fft()` instead of `rfft()`?

WARNING: The fast convolution using FFT implements *cyclic convolution*!

Take care to use *zero-padding* to increase the length of the FFT to avoid components from the cyclic convolution in the output signal.
By the way, the equation for cyclic convolution looks like this:

$$y[n] = x[n] \circledast_N h[n] = \sum_{m=0}^{N-1} x[m] \cdot h[(n-m) \bmod N].$$

If you want, you can also choose the next bigger power of two as FFT length - this is more efficient in many (but not all) cases.
After the inverse transform you should trim the result to the appropriate length.

Is the calculation with `fft_convolution()` faster than with the previously used functions?

## Using Existing Functions

Convolution is very important in many areas of signal processing, thus it is not surprising that a function for it is available in NumPy: [numpy.convolve()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.convolve.html).
This function does basically the same as our `naive_convolution()`.
Just much more efficient.

*Exercise:* Try it!

*Exercise:* Have a look at the `mode` argument.
Which "mode" did we implement before?

But that's not everything yet!

Let's import the signal processing module from SciPy and see if we find something useful there:

In [ ]:
from scipy import signal

*Exercise:* Do the same convolution as before, but now using [scipy.signal.fftconvolve()](http://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.fftconvolve.html).

TODO: longer input signal!

TODO: Have a look at the documentation, especially about the sizes of the two input arrays.
What happens if you switch the two input arrays (i.e. if you switch the input signal and the impulse response)?

TODO: `fftfilt()`:

* https://github.com/scipy/scipy/issues/1364

* https://github.com/ColumbiaCMB/kid_readout/blob/master/kid_readout/utils/fftfilt.py

* http://cnx.org/contents/9479779f-bd46-4d3a-a1d4-30580ae8aacc@10/Convolution_Algorithms

## What if the System is Non-Linear?

The function `tools.blackbox_nonlinear()` provides a non-linear system.

*Exercise:* Listen how it sounds if you send the signal from `xmas.wav` (or some other
audio signal) through this system.

*Exercise:* Check if this system can be described by an impulse response.
Use the unit impulse from before to obtain the impulse response of `tools.blackbox_nonlinear()`.

Convolve our audio signal with this impulse response and listen to it.
Do you hear any difference between the signal convolved with the impulse response and the signal sent directly through the non-linear system?

Which leads us to the final question:
Can a non-linear system be described completely with an impulse response?

## Solutions

If you had problems solving some of the exercises, don't despair!
Have a look at the [example solutions](ir-solutions.ipynb).

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>